IMPORT THE MODULES WHICH WE WILL USE FOR THE SOLUTION

In [1]:
#IMPORT THE MODULES WHICH WE WILL USE FOR THE SOLUTION

In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import itertools
from itertools import chain, combinations
import numpy as np
import re
from nltk.stem import PorterStemmer 
from collections import Counter
import math
from numpy import array
from numpy import diag
from numpy import zeros
from scipy.linalg import svd
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import inv
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import time
from operator import itemgetter
from Levenshtein import distance
from operator import itemgetter as it
from itertools import repeat
import os
import glob
import ast
from collections import defaultdict 

In [3]:
#Delete all files in tempreResults directory
filelist = glob.glob(os.path.join("./tempResults/", "*.csv"))
for f in filelist:
    os.remove(f)

GLOBAL VARIABLES

In [4]:
#GLOBAL VARIABLES

In [5]:
#percentual = 1.0

In [6]:
#CUSTOMER PROFILINGS
numberOfProfiles= 4

In [7]:
recipesFile = "real_recipe_dataset.json"

In [8]:
syntheticDataGrocery = False

In [9]:
WORDS = {}

In [10]:
stopwords =["a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are", "aren", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", "mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "won", "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", "i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've", "we'd", "we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", "would"]

In [11]:
ps = PorterStemmer()

DEFINE FUNCTIONS WHICH WILL BE USED

In [12]:
#DEFINE FUNCTIONS WHICH WILL BE USED

In [13]:
#Clean the string x
def cleaningString (x):
    result = []
    for j in x :
        j = j.lower()
        j = re.sub(r"[^a-zA-Z]+",' ',j)
        j = j.strip()
        result.append(j)
    return result

In [14]:
#Clean the string x
def cleaningStringImproved (x):
    result = []
    for j in x :
        j = j.lower()
        j = re.sub(r"[^a-zA-Z]+",' ',j)
        j = j.strip()
        result.append(j)
    return " ".join(x).strip().lower()

In [15]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s).strip())

In [16]:
#GIVEN A BASKET RETURN ALL ITEMS THAT THAT ARE RELATED TO THE GIVEN BASKET BY THE ASSOCIATION RULES
#THE ITEMS THAT ARE ALREADY IN THE BASKET WILL NOT BE RETURNED
def consequentsPrediction(inputs):
    #print("ORIGINAL" + str(inputs))    
    consequent_insieme = set()
    for left_rule in rules["antecedents"].values :      
        if left_rule.issubset(inputs):
            froz_set = rules[rules['antecedents'] == left_rule ]["consequents"].values
            test = set(itertools.chain.from_iterable(froz_set))
            for k in test :
                consequent_insieme.add(k)
    for elem in inputs:
        if elem in consequent_insieme:
            consequent_insieme.remove(elem)
    #print("PREDICTED"+ str(consequent_insieme))
    return consequent_insieme

In [17]:
#Clean the string x
def deleteStopwords (x):
    result = []
    for j in x :
        temp = j.split()
        temp2 = ""
        for i in temp:
            if i in stopwords:
                i = " "
            elif len(i) <= 2 :
                i = " "
            temp2 += i
            temp2 += " "
        temp2 = temp2.strip()
        result.append(temp2)
    return result

In [18]:
def stemming(x):
    result = []
    for j in x:
        temp = j.split()
        temp2 = ""
        for i in temp:
            stem = ps.stem(i)
            temp2 += stem
            temp2 += " "
        temp2 = temp2.strip()
        result.append(temp2)
    return result

In [19]:
def words(text): return re.findall(r'\w+', text.lower())

In [20]:
def edits_one(word):
    return_set = set()
    for w in WORDS:
        temp =  distance(word,w)
        if (temp<=1):
            return_set.add(w)
            
    return return_set

In [21]:
def edits_two(word):
    return_set = set()
    for w in WORDS:
        temp =  distance(word,w)
        if (temp<=2):
            return_set.add(w)
            
    return return_set

In [22]:
def correction(word): 
    "Most probable spelling correction for word."
    result = candidates(word)
    if len(result) == 0:
        return ""
    return max(candidates(word), key=P)

In [23]:
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or edits_one((word)) or edits_two((word)))

In [24]:
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

In [25]:
def correctionString(x):
    result = []
    for j in x:
        temp = j.split()
        temp2 = ""
        for i in temp:
            correct = correction(i)
            temp2 += correct
            temp2 += " "
        temp2 = temp2.strip()
        if(temp2 != ""):
            result.append(temp2)
    return result

# STEP 1

In [26]:
#STEP 1

Creating the two dataframes

In [27]:
#Creating the two dataframes

In [28]:
recipes = pd.DataFrame()
if syntheticDataGrocery!= True :
    
    groceries = pd.DataFrame()
    groceries = pd.read_csv("real_market_basket_dataset.csv")
    
    #Drop the columns "Item(s)" in groceries and "id" in recipes
    groceries = groceries.drop("Item(s)", axis = 1)
    stripped_nan = []
    
    for row in range(0, groceries.count()[0]):
        string_tmp_1 = pd.Series()
    
        # build column with single ingredients
        string_tmp_1 = groceries.iloc[row, :].dropna()
        string_tmp_2 = string_tmp_1.reset_index(drop = True)
        string_tmp_3 = list(string_tmp_2.to_numpy())
        stripped_nan.append(string_tmp_3)
    
    groceries = pd.DataFrame(pd.Series(stripped_nan).reset_index()).drop("index",axis=1).set_axis(["original basket"], 1, inplace = False)
    

In [29]:
recipes = pd.read_json(recipesFile) 
recipes = recipes.drop("id",axis=1)

In [30]:
start_time = time.time()

# STEP 2

In [31]:
recipes

,cuisine,ingredients
0,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,"[water, vegetable oil, wheat, salt]"
4,indian,"[black pepper, shallots, cornflour, cayenne pe..."
...,...,...
39769,irish,"[light brown sugar, granulated sugar, butter, ..."
39770,italian,"[KRAFT Zesty Italian Dressing, purple onion, b..."
39771,irish,"[eggs, citrus fruit, raisins, sourdough starte..."
39772,chinese,"[boneless chicken skinless thigh, minced garli..."


In [32]:
#COLLECT ALL UNIQUE CUISINE NAMES
cuisine_name = []
for unique_name in recipes.cuisine.unique():
    cuisine_name.append(unique_name)

In [33]:
groceries

,original basket
0,"[citrus fruit, semi-finished bread, margarine,..."
1,"[tropical fruit, yogurt, coffee]"
2,[whole milk]
3,"[pip fruit, yogurt, cream cheese, meat spreads]"
4,"[other vegetables, whole milk, condensed milk,..."
...,...
9830,"[sausage, chicken, beef, hamburger meat, citru..."
9831,[cooking chocolate]
9832,"[chicken, citrus fruit, other vegetables, butt..."
9833,"[semi-finished bread, bottled water, soda, bot..."


In [34]:
#NORMALIZE THE DATA

In [35]:
lista_prova = [cleaningString(s) for s in recipes["ingredients"]]
recipes["ingredients"] = lista_prova

In [36]:
lista_prova = [cleaningString(s) for s in groceries["original basket"]]
groceries["original basket"] = lista_prova

In [37]:
#DELETE STOPWORDS AND 2-GRAM


In [38]:
lista_prova = [deleteStopwords(s) for s in recipes["ingredients"]]
recipes["ingredients"] = lista_prova

In [39]:
lista_prova = [deleteStopwords(s) for s in groceries["original basket"]]
groceries["original basket"] = lista_prova

In [40]:
#LEVENSTHEIN DISTANCE

In [41]:
recipes

,cuisine,ingredients
0,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,"[water, vegetable oil, wheat, salt]"
4,indian,"[black pepper, shallots, cornflour, cayenne pe..."
...,...,...
39769,irish,"[light brown sugar, granulated sugar, butter, ..."
39770,italian,"[kraft zesty italian dressing, purple onion, b..."
39771,irish,"[eggs, citrus fruit, raisins, sourdough starte..."
39772,chinese,"[boneless chicken skinless thigh, minced garli..."


In [42]:
temporary = " ".join(list(itertools.chain.from_iterable(recipes["ingredients"])))

In [43]:
WORDS = Counter(words(temporary))

In [44]:
#temporary location, need to move this function in the function section
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

In [45]:
lista_prova = [correctionString(s) for s in groceries["original basket"]]
groceries["original basket"] = lista_prova

In [46]:
#STEMMING

In [47]:
#stemming ingredients
lista_prova=[]
for recipe in recipes["ingredients"]:
    lista_prova.append(stemming(recipe))
recipes["ingredients"] = lista_prova

In [48]:
#stemming groceries
lista_prova=[]
for grocery in groceries["original basket"] :
    lista_prova.append(stemming(grocery))
groceries["original basket"] = lista_prova

In [49]:
recipes

,cuisine,ingredients
0,greek,"[romain lettuc, black oliv, grape tomato, garl..."
1,southern_us,"[plain flour, ground pepper, salt, tomato, gro..."
2,filipino,"[egg, pepper, salt, mayonais, cook oil, green ..."
3,indian,"[water, veget oil, wheat, salt]"
4,indian,"[black pepper, shallot, cornflour, cayenn pepp..."
...,...,...
39769,irish,"[light brown sugar, granul sugar, butter, warm..."
39770,italian,"[kraft zesti italian dress, purpl onion, brocc..."
39771,irish,"[egg, citru fruit, raisin, sourdough starter, ..."
39772,chinese,"[boneless chicken skinless thigh, minc garlic,..."


In [50]:
#FOR EACH RECIPE COLLECT AND STORE ALL DOCUMENTS OF THAT RECIPE IN STRING FORMAT
dict_recipes_true_name = {}
for recipe_name in cuisine_name :
    recipe_df = recipes.loc[recipes['cuisine'] == recipe_name]
    array = recipe_df["ingredients"].values
    doc = []
    for list_recipe in array:
        string = ""
        string = ", ".join(list_recipe)
        string = string.strip()
        doc.append(string)
    dict_recipes_true_name[recipe_name] = doc

In [51]:
groceries

,original basket
0,"[citru fruit, semi bread, margarin, readi soup]"
1,"[tropic fruit, yogurt, coffe]"
2,[whole milk]
3,"[pie fruit, yogurt, cream chees, meat spread]"
4,"[veget, whole milk, condens milk, long lime bake]"
...,...
9830,"[sausag, chicken, beef, hamburg meat, citru fr..."
9831,[cook chocol]
9832,"[chicken, citru fruit, veget, butter, yogurt, ..."
9833,"[semi bread, bottl water, soda, bottl beer]"


In [52]:
stripped_nan = groceries["original basket"].values

# STEP 3

ONE HOT ENCODING BY MLXTEND

In [53]:
#ONE HOT ENCODING BY MLXTEND

In [54]:
te = TransactionEncoder()
te_ary = te.fit(stripped_nan).transform(stripped_nan)
one_hot_encoded = pd.DataFrame(te_ary, columns = te.columns_)

THE DATA IS READY TO BE APPLIED TO A-PRIORI ALGORITHM. THE MINIMUM SUPPORT IS 1%

In [55]:
#THE DATA IS READY TO BE APPLIED TO A-PRIORI ALGORITHM. THE MINIMU SUPPORT IS 0.01, THAT IS 1%
results_apriori = apriori(one_hot_encoded, min_support=0.01,
                          use_colnames=True, verbose = 0)

GET ALL THE ASSOCIATION RULES FROM THE RESULT OF THE A-PRIORI ALGORITHM  WITH CONFIDENCE ABOVE 0.5.

In [56]:
#GET ALL THE ASSOCIATION RULES FROM THE RESULT OF THE A-PRIORI ALGORITHM  WITH CONFIDENCE ABOVE 0.5.
rules = association_rules(results_apriori, metric = 'confidence', 
                                      min_threshold = 0.5)

In [57]:
#total_baskets = len(groceries)

In [58]:
#total_baskets

In [59]:
#rules["antecedentsCounts"] = rules["antecedent support"] * total_baskets

In [60]:
#rules["consequentCounts"] = rules["consequent support"] * total_baskets

In [61]:
#rules["supportCount"] = rules["support"] * total_baskets

In [62]:
#rules = rules.astype({"antecedentsCounts": int, "consequentCounts": int, "supportCount": int })

In [63]:
rules["interest"] = rules["confidence"] - rules["consequent support"]

In [64]:
#rules["interest"] = rules["confidence"] - (rules["consequentCounts"]/total_baskets)

In [65]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,interest
0,"(butter, veget)",(whole milk),0.020031,0.255516,0.011490,0.573604,2.244885,0.006371,1.745992,0.318088
1,"(root veget, citru fruit)",(veget),0.017692,0.193493,0.010371,0.586207,3.029608,0.006948,1.949059,0.392714
2,"(curd, yogurt)",(whole milk),0.017285,0.255516,0.010066,0.582353,2.279125,0.005649,1.782567,0.326837
3,"(veget, egg)",(whole milk),0.022267,0.255516,0.012303,0.552511,2.162336,0.006613,1.663694,0.296995
4,"(veget, pie fruit)",(whole milk),0.026131,0.255516,0.013523,0.517510,2.025351,0.006846,1.543003,0.261994
5,"(roll bun, root veget)",(veget),0.024301,0.193493,0.012201,0.502092,2.594890,0.007499,1.619792,0.308599
6,"(roll bun, root veget)",(whole milk),0.024301,0.255516,0.012710,0.523013,2.046888,0.006500,1.560804,0.267497
7,"(tropic fruit, root veget)",(veget),0.021047,0.193493,0.012303,0.584541,3.020999,0.008231,1.941244,0.391048
8,"(tropic fruit, root veget)",(whole milk),0.021047,0.255516,0.011998,0.570048,2.230969,0.006620,1.731553,0.314532
9,"(yogurt, root veget)",(veget),0.025826,0.193493,0.012913,0.500000,2.584078,0.007916,1.613015,0.306507


In [66]:
#GET ALL THE ASSOCIATION RULES WITH LIFT > 1 . ALL ASSOCIATION RULES COLLECTED SO FAR CONFIDENCE =>0.5
#LIFT > 1.

In [67]:
#rules = rules[ (rules['lift'] > 1) ]

In [68]:
rules = rules[ (rules['interest'] >= 0.35) ]

In [69]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,interest
1,"(root veget, citru fruit)",(veget),0.017692,0.193493,0.010371,0.586207,3.029608,0.006948,1.949059,0.392714
7,"(tropic fruit, root veget)",(veget),0.021047,0.193493,0.012303,0.584541,3.020999,0.008231,1.941244,0.391048


In [70]:
pesol = rules.drop(["antecedent support", "consequent support", "lift", "leverage", "conviction"], axis = 1)

In [71]:
pesol

,antecedents,consequents,support,confidence,interest
1,"(root veget, citru fruit)",(veget),0.010371,0.586207,0.392714
7,"(tropic fruit, root veget)",(veget),0.012303,0.584541,0.391048


In [72]:
items = sorted(
    set(itertools.chain.from_iterable(rules["antecedents"].values)))

In [73]:
groceries

,original basket
0,"[citru fruit, semi bread, margarin, readi soup]"
1,"[tropic fruit, yogurt, coffe]"
2,[whole milk]
3,"[pie fruit, yogurt, cream chees, meat spread]"
4,"[veget, whole milk, condens milk, long lime bake]"
...,...
9830,"[sausag, chicken, beef, hamburg meat, citru fr..."
9831,[cook chocol]
9832,"[chicken, citru fruit, veget, butter, yogurt, ..."
9833,"[semi bread, bottl water, soda, bottl beer]"


In [74]:
"""
import numpy as np
test = groceries.apply(np.vectorize(consequentsPrediction), axis=0).set_axis(["predicted items"], 1, inplace = False)
"""

'\nimport numpy as np\ntest = groceries.apply(np.vectorize(consequentsPrediction), axis=0).set_axis(["predicted items"], 1, inplace = False)\n'

In [75]:
predicted_baskets = []
total_baskets = []
for i in range(len(groceries["original basket"])):
    result = list(consequentsPrediction(groceries["original basket"][i]))
    predicted_baskets.append(result)
    total_baskets.append(groceries["original basket"][i]+result)

In [76]:
groceries["association items"] = pd.Series(predicted_baskets).reset_index().drop("index",1)

In [77]:
groceries["original basket + association items"] = pd.Series(total_baskets).reset_index().drop("index",1)

In [78]:
groceries

,original basket,association items,original basket + association items
0,"[citru fruit, semi bread, margarin, readi soup]",[],"[citru fruit, semi bread, margarin, readi soup]"
1,"[tropic fruit, yogurt, coffe]",[],"[tropic fruit, yogurt, coffe]"
2,[whole milk],[],[whole milk]
3,"[pie fruit, yogurt, cream chees, meat spread]",[],"[pie fruit, yogurt, cream chees, meat spread]"
4,"[veget, whole milk, condens milk, long lime bake]",[],"[veget, whole milk, condens milk, long lime bake]"
...,...,...,...
9830,"[sausag, chicken, beef, hamburg meat, citru fr...",[veget],"[sausag, chicken, beef, hamburg meat, citru fr..."
9831,[cook chocol],[],[cook chocol]
9832,"[chicken, citru fruit, veget, butter, yogurt, ...",[],"[chicken, citru fruit, veget, butter, yogurt, ..."
9833,"[semi bread, bottl water, soda, bottl beer]",[],"[semi bread, bottl water, soda, bottl beer]"


# STEP 4

In [79]:
#STEP 4

COLLECT ALL UNIQUE CUISINE NAMES

FOR EACH RECIPE COLLECT AND STORE ALL DOCUMENTS OF THAT RECIPE IN STRING FORMAT

In [80]:
#FOR EACH RECIPE COLLECT AND STORE ALL DOCUMENTS OF THAT RECIPE IN STRING FORMAT
"""
dict_string = {}
for recipe_name in cuisine_name :
    recipe_df = recipes.loc[recipes['cuisine'] == recipe_name]
    array = recipe_df["ingredients"].values
    doc = ""
    for list_recipe in array:
        for elem in list_recipe :
            doc += elem + " "
    doc = doc[:-1]
    dict_string[recipe_name] = doc
"""

'\ndict_string = {}\nfor recipe_name in cuisine_name :\n    recipe_df = recipes.loc[recipes[\'cuisine\'] == recipe_name]\n    array = recipe_df["ingredients"].values\n    doc = ""\n    for list_recipe in array:\n        for elem in list_recipe :\n            doc += elem + " "\n    doc = doc[:-1]\n    dict_string[recipe_name] = doc\n'

CREATE THE TERM-DOCUMENT MATRIX

In [81]:
#CREATE THE TERM-DOCUMENT MATRIX

CREATE THE DATAFRAME WHICH REPRESENTS THE TF-IDF MATRIX

In [82]:
#CREATE THE DATAFRAME WHICH REPRESENTS THE TF-IDF MATRIX
#matrix = pd.DataFrame(matrixWeights(corpus_R, "test")).T

In [83]:
#matrix = matrix.fillna(0.0)

In [84]:
#matrix.columns = documents_label

In [85]:
#matrix

# TEST AREA

In [86]:
#FOR EACH RECIPE COLLECT AND STORE ALL DOCUMENTS OF THAT RECIPE IN STRING FORMAT
dict_recipes = {}
for recipe_name in cuisine_name :
    recipe_df = recipes.loc[recipes['cuisine'] == recipe_name]
    array = recipe_df["ingredients"].values
    doc = []
    for list_recipe in array:
        string = ""
        for elem in list_recipe :
            string += elem + " "
        string = string.strip()
        doc.append(string)
    dict_recipes[recipe_name] = doc

In [87]:
#START OF TEST AREA

In [88]:
#END OF TEST AREA

In [89]:
groceries

,original basket,association items,original basket + association items
0,"[citru fruit, semi bread, margarin, readi soup]",[],"[citru fruit, semi bread, margarin, readi soup]"
1,"[tropic fruit, yogurt, coffe]",[],"[tropic fruit, yogurt, coffe]"
2,[whole milk],[],[whole milk]
3,"[pie fruit, yogurt, cream chees, meat spread]",[],"[pie fruit, yogurt, cream chees, meat spread]"
4,"[veget, whole milk, condens milk, long lime bake]",[],"[veget, whole milk, condens milk, long lime bake]"
...,...,...,...
9830,"[sausag, chicken, beef, hamburg meat, citru fr...",[veget],"[sausag, chicken, beef, hamburg meat, citru fr..."
9831,[cook chocol],[],[cook chocol]
9832,"[chicken, citru fruit, veget, butter, yogurt, ...",[],"[chicken, citru fruit, veget, butter, yogurt, ..."
9833,"[semi bread, bottl water, soda, bottl beer]",[],"[semi bread, bottl water, soda, bottl beer]"


In [90]:
recipes

,cuisine,ingredients
0,greek,"[romain lettuc, black oliv, grape tomato, garl..."
1,southern_us,"[plain flour, ground pepper, salt, tomato, gro..."
2,filipino,"[egg, pepper, salt, mayonais, cook oil, green ..."
3,indian,"[water, veget oil, wheat, salt]"
4,indian,"[black pepper, shallot, cornflour, cayenn pepp..."
...,...,...
39769,irish,"[light brown sugar, granul sugar, butter, warm..."
39770,italian,"[kraft zesti italian dress, purpl onion, brocc..."
39771,irish,"[egg, citru fruit, raisin, sourdough starter, ..."
39772,chinese,"[boneless chicken skinless thigh, minc garlic,..."


In [91]:
lista_prova = [listToString(s) for s in groceries["original basket + association items"]]
groceries["document basket"] = lista_prova

In [92]:
lista_prova = [listToString(s) for s in recipes["ingredients"]]
recipes["ingredients"] = lista_prova

In [93]:
groceries.drop(["original basket", "association items", "original basket + association items"], axis = 1)

,document basket
0,citru fruit semi bread margarin readi soup
1,tropic fruit yogurt coffe
2,whole milk
3,pie fruit yogurt cream chees meat spread
4,veget whole milk condens milk long lime bake
...,...
9830,sausag chicken beef hamburg meat citru fruit g...
9831,cook chocol
9832,chicken citru fruit veget butter yogurt frozen...
9833,semi bread bottl water soda bottl beer


In [94]:
groceries_documents = groceries["document basket"].values.tolist()

In [95]:
chunks =500

In [96]:
chunks_documents = [groceries_documents[i:i + chunks] for i in range(0, len(groceries_documents), chunks)]

In [97]:
len(chunks_documents)

20

In [98]:
def createAndSaveDataframeImproved(input_documents, temp_name):
    n = 12
    
    final_dataframe = pd.DataFrame()
    for recipe_name in cuisine_name:
        cv=CountVectorizer()
        tfidf_transformer=TfidfTransformer(norm= None ,smooth_idf=False,use_idf=True,sublinear_tf=False)
        #GET THE TF-IDF OF THE X MATRIX
        #count
        word_count_vector=cv.fit_transform(dict_recipes[recipe_name])
        list_count = word_count_vector.toarray()
        #check no division by 0
        b = list_count.max(axis=1)[:,None]
        b[b==0] = 1
        
        resulting_count = list_count / b
        #learn the idf
        tf_idf = tfidf_transformer.fit(resulting_count)        
        tf_idf.idf_ = tf_idf.idf_ - 1
        X = tf_idf.transform(resulting_count)
        
        #GET THE TF-IDF OF THE Y MATRIX
        
        word_count_vector_y = cv.transform(input_documents)
        word_count_vector_y_array = word_count_vector_y.toarray()
        #Check no division by 0
        c = word_count_vector_y_array.max(axis=1)[:,None]
        c[c==0] = 1
        word_count_vector_y_result = word_count_vector_y_array / c
        Y = tf_idf.transform(word_count_vector_y_result)
        
        #vectorizer = TfidfVectorizer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)
        #X = vectorizer.fit_transform(dict_recipes[recipe_name])
        #Y = vectorizer.transform(input_documents)
        # iterate through rows:
        cosine_result = cosine_similarity(X,Y)
        dataframe_cosine = pd.DataFrame(cosine_result)
        
        #dataframe_cosine = dataframe_cosine.loc[~(dataframe_cosine==0).all(axis=1)]
        
        s = dataframe_cosine.T
        s["index"] = s.index
        cols = s.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        s=s[cols]
        
        list_cuisine=[]
        for i in range(n):
            list_cuisine.append(recipe_name)
        new_d = [list(zip(repeat(row["index"]), map(it(0), sorted(row[1:].iteritems(), key=it(1) ,reverse=1)[:n]),list_cuisine))
                    for _, row  in s.iterrows()]
        

        result_cosine = pd.DataFrame([[( input_documents[int(i)] ,cuisine, dict_recipes_true_name[recipe_name][j],dataframe_cosine[i][j]) for i,j,cuisine in s] for s in new_d]).T
        final_dataframe = pd.DataFrame.append(final_dataframe,result_cosine)
        
    
    temp_list = [sorted(s,key=itemgetter(3), reverse = True)[:10] for s in final_dataframe.T.values.tolist()]
    final_dataframe= pd.DataFrame(temp_list)
    final_dataframe.to_csv(temp_name, index=False)

In [99]:
index = 0
input_dataframes = []
for i in chunks_documents:
    temp_name = "./tempResults/"+str(index)+".csv"
    createAndSaveDataframeImproved(i, temp_name)
    #input_dataframes.append(createAndSaveDataframe(i))
    index+=1
#appended_data = pd.concat(input_dataframes)
#appended_data.to_csv("test.csv")

In [100]:
#Collect all the csv files generated in tempResults
extension = 'csv'
#Be warned that all_filenames is disorganized list of the files 
all_filenames = [i for i in glob.glob('./tempResults/*.{}'.format(extension))]

In [101]:
all_filenames

['./tempResults\\0.csv',
 './tempResults\\1.csv',
 './tempResults\\10.csv',
 './tempResults\\11.csv',
 './tempResults\\12.csv',
 './tempResults\\13.csv',
 './tempResults\\14.csv',
 './tempResults\\15.csv',
 './tempResults\\16.csv',
 './tempResults\\17.csv',
 './tempResults\\18.csv',
 './tempResults\\19.csv',
 './tempResults\\2.csv',
 './tempResults\\3.csv',
 './tempResults\\4.csv',
 './tempResults\\5.csv',
 './tempResults\\6.csv',
 './tempResults\\7.csv',
 './tempResults\\8.csv',
 './tempResults\\9.csv']

In [102]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "resultTFIDF.csv", index=False)

In [103]:
#math.log(5/1) IDF use ln_e i guess?

In [104]:
conv = {}
for i in range(10):
    conv[i] = ast.literal_eval

In [105]:
conv

{0: <function ast.literal_eval(node_or_string)>,
 1: <function ast.literal_eval(node_or_string)>,
 2: <function ast.literal_eval(node_or_string)>,
 3: <function ast.literal_eval(node_or_string)>,
 4: <function ast.literal_eval(node_or_string)>,
 5: <function ast.literal_eval(node_or_string)>,
 6: <function ast.literal_eval(node_or_string)>,
 7: <function ast.literal_eval(node_or_string)>,
 8: <function ast.literal_eval(node_or_string)>,
 9: <function ast.literal_eval(node_or_string)>}

In [106]:
#combined_csv = pd.read_csv("resultTFIDF.csv", converters={"0": ast.literal_eval, "1": ast.literal_eval, "0": ast.literal_eval})
combined_csv = pd.read_csv("resultTFIDF.csv", converters=conv)

In [107]:
combined_csv

,0,1,2,3,4,5,6,7,8,9
0,"(citru fruit semi bread margarin readi soup, k...","(citru fruit semi bread margarin readi soup, t...","(citru fruit semi bread margarin readi soup, k...","(citru fruit semi bread margarin readi soup, j...","(citru fruit semi bread margarin readi soup, t...","(citru fruit semi bread margarin readi soup, k...","(citru fruit semi bread margarin readi soup, v...","(citru fruit semi bread margarin readi soup, f...","(citru fruit semi bread margarin readi soup, f...","(citru fruit semi bread margarin readi soup, f..."
1,"(tropic fruit yogurt coffe, french, larg egg y...","(tropic fruit yogurt coffe, mexican, oliv oil,...","(tropic fruit yogurt coffe, greek, coffe, suga...","(tropic fruit yogurt coffe, british, cream, co...","(tropic fruit yogurt coffe, italian, tropic fr...","(tropic fruit yogurt coffe, indian, water, coc...","(tropic fruit yogurt coffe, cajun_creole, wate...","(tropic fruit yogurt coffe, southern_us, milk,...","(tropic fruit yogurt coffe, indian, cardamom p...","(tropic fruit yogurt coffe, filipino, fruit, w..."
2,"(whole milk, russian, lemon, salt, whole milk,...","(whole milk, indian, lemon juic, whole milk, 0...","(whole milk, indian, whole milk, lemon juic, s...","(whole milk, indian, salt, lemon juic, whole m...","(whole milk, indian, whole milk, lime, 0.77353...","(whole milk, indian, fresh lemon juic, whole m...","(whole milk, indian, fresh lemon juic, whole m...","(whole milk, southern_us, sugar, salt, bake po...","(whole milk, french, larg egg, sugar, purpos f...","(whole milk, french, whole milk, unsalt butter..."
3,"(pie fruit yogurt cream chees meat spread, fil...","(pie fruit yogurt cream chees meat spread, ind...","(pie fruit yogurt cream chees meat spread, sou...","(pie fruit yogurt cream chees meat spread, fil...","(pie fruit yogurt cream chees meat spread, rus...","(pie fruit yogurt cream chees meat spread, spa...","(pie fruit yogurt cream chees meat spread, bri...","(pie fruit yogurt cream chees meat spread, mex...","(pie fruit yogurt cream chees meat spread, sou...","(pie fruit yogurt cream chees meat spread, rus..."
4,"(veget whole milk condens milk long lime bake,...","(veget whole milk condens milk long lime bake,...","(veget whole milk condens milk long lime bake,...","(veget whole milk condens milk long lime bake,...","(veget whole milk condens milk long lime bake,...","(veget whole milk condens milk long lime bake,...","(veget whole milk condens milk long lime bake,...","(veget whole milk condens milk long lime bake,...","(veget whole milk condens milk long lime bake,...","(veget whole milk condens milk long lime bake,..."
...,...,...,...,...,...,...,...,...,...,...
9830,(frankfurt tropic fruit veget roll bun bottl w...,(frankfurt tropic fruit veget roll bun bottl w...,(frankfurt tropic fruit veget roll bun bottl w...,(frankfurt tropic fruit veget roll bun bottl w...,(frankfurt tropic fruit veget roll bun bottl w...,(frankfurt tropic fruit veget roll bun bottl w...,(frankfurt tropic fruit veget roll bun bottl w...,(frankfurt tropic fruit veget roll bun bottl w...,(frankfurt tropic fruit veget roll bun bottl w...,(frankfurt tropic fruit veget roll bun bottl w...
9831,"(hamburg meat veget pickl veget, russian, fres...","(hamburg meat veget pickl veget, mexican, vege...","(hamburg meat veget pickl veget, chinese, soy ...","(hamburg meat veget pickl veget, mexican, garl...","(hamburg meat veget pickl veget, mexican, red ...","(hamburg meat veget pickl veget, southern_us, ...","(hamburg meat veget pickl veget, mexican, pork...","(hamburg meat veget pickl veget, russian, whit...","(hamburg meat veget pickl veget, irish, veget ...","(hamburg meat veget pickl veget, mexican, wate..."
9832,"(white wine, french, dri white wine, salt, whi...","(white wine, thai, fish sauc, lime juic, soy s...","(white wine, french, black peppercorn, white w...","(white wine, indian, chili flake, red bell pep...","(white wine, indian, port wine, fresh ginger, ...","(white wine, thai, oliv oil, 

In [108]:
#combined_csv2 = pd.read_csv("resultTFIDFStem.csv", converters={"0": ast.literal_eval})

In [109]:
#Input : index of the element, the most N common in the column
def customerProfiling(index, index_common):
    out = [recipe for basket,cuisine,recipe,cosine in combined_csv[""+str(index)+""] ] 
    recipes_common = Counter(out)
    recipes_looking, value_cos = recipes_common.most_common(40)[index_common]
    output = defaultdict(float) 
    for b, c, r, cos in combined_csv[""+str(index)+""]: 
        output[r] += cos 
    return [(recipe, cos/value_cos) for recipe,cos in list(output.items()) if recipe == recipes_looking] 

In [110]:
customerProfiling(1,0)

[('milk, purpos flour, cook oil, salt, larg egg, beer', 0.613806976012385)]

In [111]:
totalProfiles= []
for n in range(numberOfProfiles):
    customerProfiles = []
    for i in range(4):
        result = customerProfiling(i,n)[0]
        customerProfiles.append(result)
    totalProfiles.append(customerProfiles)

In [112]:
totalProfiles

[[('butter, yeast, purpos flour, salt, sugar, beer', 0.6101406967573048),
  ('milk, purpos flour, cook oil, salt, larg egg, beer', 0.613806976012385),
  ('ground black pepper, garlic, dri thyme, butter, carrot, can low sodium chicken broth, bake potato, salt, veget, heavi cream, onion',
   0.5515347423961969),
  ('soy sauc, green onion, long grain rice, chuck roast, veget oil, water, sesam oil, toast sesam seed, lettuc leav, beer',
   0.5227788362144279)],
 [('water, coconut, sugar, salt, fruit, rice flour', 0.6193049272802615),
  ('vanilla, oil, whole milk, cream chees, roll, sour cream, whip cream, condens milk',
   0.5164836594073257),
  ('milk, purpos flour, cook oil, salt, larg egg, beer', 0.5243538000384562),
  ('marmit, onion, chees, bun, toast', 0.48745347990869076)],
 [('vanilla, oil, whole milk, cream chees, roll, sour cream, whip cream, condens milk',
   0.5374519818510719),
  ('water, coconut, sugar, salt, fruit, rice flour', 0.5903719551567298),
  ('buttermilk, whole wheat

In [113]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 436.02601504325867 seconds ---


In [114]:
#Delete all files in tempreResults directory
filelist = glob.glob(os.path.join("./tempResults/", "*.csv"))
for f in filelist:
    os.remove(f)

# Create finalReport

In [115]:
f=open('finalReport.txt','w')
index_write = 1
f.write("Customer profiling based on cosine similarity score\n")
f.write("Max number of recipes per profile: 4\n\n\n\n")
for profile in totalProfiles:
    index_recipe = 1
    f.write("Customer Profile "+ str(index_write))
    f.write("\n")
    f.write("We have customers who plan to make the following recipes :\n")
    for elem in profile:
        recipe, cosine = elem
        f.write(str(index_recipe) + ") " + recipe + " (Avg.Score: "+ str(cosine)+ ")\n")
        index_recipe+=1
    f.write("\n\n\n\n")
    index_write+=1
f.close()